In [1]:
from document_processor import DocumentProcessor
from knowledge_graph import KnowledgeGraph
from vector_store import VectorStore
from entity_extraction import GraphEntityRetriever
from retrieval import Retriever
from rag_chain import RagChain
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings("ignore")
load_dotenv()

required_vars = ["NEO4J_URI", "NEO4J_USERNAME", "NEO4J_PASSWORD", "OPENROUTER_API_KEY"]
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    print(f"Warning: The following environment variables are not set: {', '.join(missing_vars)}")
    print("Please set them in a .env file or directly in the environment.")
else:
    print("All required environment variables are set.")

All required environment variables are set.


In [2]:
# Initialize the System

document_processor = DocumentProcessor()
knowledge_graph = KnowledgeGraph()
vector_store = VectorStore(knowledge_graph)
entity_extractor = GraphEntityRetriever()
retriever = Retriever(knowledge_graph, vector_store)
rag_chain = RagChain(retriever)



# Load the documents

In [3]:
documents = document_processor.process_documents()
print(f"Loaded {len(documents)} document chunks from the knowledge base")
print("\nFirst document chunk:")
print(documents[0].page_content[:500] + "...")

Loaded 78 document chunks from the knowledge base

First document chunk:
**Chris Olande: A Deep Professional Profile for RAG Systems**

**Overview**

Chris Olande is an statistics and programming student at Kenyatta University in Nairobi, Kenya. With a rigorous academic foundation in statistics and programming, Chris is cultivating a multidisciplinary approach that intersects applied mathematics, artificial intelligence, and human-centered design. He brings together analytical precision, computational depth, and social responsibility in his work. Chris is not only bu...


# Build the knowledge graph

In [4]:
from IPython.display import clear_output

# Create or load an existing knowledge graph from the documents
knowledge_graph.load_or_create_graph(documents)
clear_output()

In [5]:
#knowledge_graph.get_graph_summary()

In [6]:
knowledge_graph.visualize_graph()

GraphWidget(layout=Layout(height='800px', width='100%'))

# Test entity Extraction

In [7]:
question = "Who is Chris Olande?"
entity = entity_extractor.extract_entities(question)
print(entity)

Extracting entities for question: Who is Chris Olande?
Extracted 1 entities: ['Chris Olande']
['Chris Olande']


# Test structured entity extraction

In [8]:
structured_results = entity_extractor.structured_retriever(question)

print(f"Question: {question}")
print("\nStructured retrieval results:")
print(structured_results)

Processing structured retrieval for question: Who is Chris Olande?
Extracting entities for question: Who is Chris Olande?
Extracted 1 entities: ['Chris Olande']
Retrieved 31 relationships for entity 'Chris Olande'
Found 31 relationships for entity 'Chris Olande'
  - Chris Olande - STUDENT_AT -> Kenyatta University
  - Chris Olande - USES -> Python
  - Chris Olande - USES -> LangChain
  - Chris Olande - USES -> Hugging Face pipelines
  - Chris Olande - USES -> OpenRouter’s LLaMA 3.2B and Gemini embeddings
  ... and 26 more
Question: Who is Chris Olande?

Structured retrieval results:
Chris Olande - STUDENT_AT -> Kenyatta University
Chris Olande - USES -> Python
Chris Olande - USES -> LangChain
Chris Olande - USES -> Hugging Face pipelines
Chris Olande - USES -> OpenRouter’s LLaMA 3.2B and Gemini embeddings
Chris Olande - USES -> FAISS or Pinecone
Chris Olande - DESIGNS -> intelligent job search assistant
Chris Olande - LOCATED_IN -> Africa
Chris Olande - PROMOTES -> STEM exposure and en

# Create the hybrid vector index

In [9]:
vector_store.create_hybrid_index()

print("Hybrid vector index created successfully!")

Hybrid vector index created successfully!


# Create vector index

In [10]:
# create vector index from documents
vector_store.create_vector_index(documents)

print("Vector index created successfully!")

Vector index created successfully!


## Test vector Retrieval

In [11]:
# Test vector retrieval
vector_results = retriever.vector_retrieval(question, k=2)

print(f"Question: {question}")
print("\nVector retrieval results:")
for i, doc in enumerate(vector_results):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:300] + "...")

Question: Who is Chris Olande?

Vector retrieval results:

Document 1:
 in statistics and machine learning, explaining complex concepts like backpropagation, gradient descent, and dimensionality reduction using intuitive analogies and real-life examples.

**Personality, Work Ethic, and Personal Interests**

Chris is disciplined, curious, and mathematically inclined. He...

Document 2:
**Chris Olande: A Deep Professional Profile for RAG Systems**

**Overview**

Chris Olande is an statistics and programming student at Kenyatta University in Nairobi, Kenya. With a rigorous academic foundation in statistics and programming, Chris is cultivating a multidisciplinary approach that inter...

Document 3:
.
Have we anything new to offer upon the subject?  Nothing.  We have held the
subject up in every light of which it is capable; but it has been all in vain.
Shall we resort to entreaty and humble supplication?  What terms shall we
find which have not been already exhausted?  Let us not, I besee

# Test hybrid retrieval


In [12]:
# Test hybrid retrieval
hybrid_results = retriever.hybrid_retrieval(question)

print(f"Question: {question}")
print("\nHybrid retrieval results:")
print(hybrid_results)

Question: Who is Chris Olande?

Hybrid retrieval results:
Structured data:
            No relevant information found in the knowledge graph.

            Unstructured data:
             in statistics and machine learning, explaining complex concepts like backpropagation, gradient descent, and dimensionality reduction using intuitive analogies and real-life examples.

**Personality, Work Ethic, and Personal Interests**

Chris is disciplined, curious, and mathematically inclined. He can spend hours discussing abstract mathematical theory—from convergence of sequences to information entropy. Despite his technical prowess, he remains grounded and people-centered, embodying humility and empathy in all collaborations.

* **Personality traits**: Methodical, introspective, self-motivated, collaborative, and principled.
* **Coding style**: Clean, modular, well-documented with adherence to PEP-8 and best practices in version control.
* **Personal vision**: To become one of Africa’s leading data 

# Test RAG Chain

In [13]:
from typing import Optional, List, Tuple

# Global chat history
chat_history: List[Tuple[str, str]] = []

def ask_questions(
    rag_chain,
    question: str,
    chat_history_override: Optional[List[Tuple[str, str]]] = None
):
    global chat_history

    if chat_history_override is None:
        chat_history = []
    else:
        chat_history = chat_history_override

    answer = rag_chain.invoke({
        "question": question,
        "chat_history": chat_history
    })

    chat_history.append((question, answer))

    print(f"Question: {question}")
    print(f"\nAnswer: {answer}\n")

    return answer


In [14]:
answer1 = ask_questions(rag_chain = rag_chain, question = question, chat_history_override = None)

Question: Who is Chris Olande?

Answer:  Chris Olande is a statistics and programming student at Kenyatta University in Nairobi, Kenya. He is cultivating a multidisciplinary approach that intersects applied mathematics, artificial intelligence, and human-centered design. Chris is known for his analytical precision, computational depth, and social responsibility in his work. He is passionate about building machine learning systems and shaping the future of AI literacy and ethical, context-aware data applications in Kenya and beyond. He is preparing for a future that merges AI systems research with applied data science in dynamic sectors such as intelligent agents, AI in education, research labs & AI startups, and open-source contributions.



## Test using a follow up question

In [15]:
question = "What does he even do at Kenyatta University?"
answer2 = ask_questions(rag_chain = rag_chain, question = question, chat_history_override = chat_history)


Question: What does he even do at Kenyatta University?

Answer:  Chris Olande is a student at Kenyatta University in Nairobi, Kenya. He is not specified to have a faculty role, but he is known for tutoring peers in statistics and machine learning, and he is also involved in educational leadership and STEM advocacy, such as leading a pioneering educational visit for Grade 7 students to Impala Glass Industries in Nairobi.



In [16]:

question = "How and what tools does he use, whats his tech stack?"
answer3 = ask_questions(rag_chain = rag_chain, question = question, chat_history_override = chat_history)


Question: How and what tools does he use, whats his tech stack?

Answer:  Chris Olande uses a variety of tools and technologies in his studies and projects at Kenyatta University. These include:

* Programming languages: Python, R
* Data science libraries: NumPy, pandas, matplotlib, seaborn, scikit-learn, SciPy
* Deep learning and NLP: PyTorch, Hugging Face Transformers, Fastai, OpenCV, LangChain, SentenceTransformers
* ML Experiment Tracking: Weights & Biases, TensorBoard
* Deployment Tools: Streamlit, Gradio, Flask, Docker
* Version control: Git/GitHub
* Shell scripting for automated workflows
* Data preprocessing and augmentation: OpenCV, torchvision transforms
* Vector search: FAISS or Pinecone
* AI and language model interpretability, ethical AI, or real-time AI tooling (potential contributions in research labs & AI startups)
* LangChain Expression Language (LCEL) for agentic RAG systems
* OpenRouter’s LLaMA 3.2B and Gemini embeddings for an intelligent job search assistant (futur

# Test questions based on romeo and juliet knowledge base

In [17]:
question = "give me a brief story about the romeo and juliet book"

answer4 = ask_questions(rag_chain = rag_chain, question = question)



Question: give me a brief story about the romeo and juliet book

Answer:  "Romeo and Juliet" is a tragic play written by William Shakespeare. Set in Verona, the story revolves around two young lovers, Romeo Montague and Juliet Capulet, who belong to two rival families. Despite their families' longstanding feud, Romeo and Juliet fall in love at first sight and secretly marry with the help of Friar Lawrence. However, their happiness is short-lived as their marriage is discovered, leading to a series of unfortunate events. Romeo is banished from Verona, and Juliet is forced to marry Paris, a kinsman of the Prince, to end the feud.

In a desperate attempt to be reunited with Romeo, Juliet is given a potion by Friar Lawrence that is supposed to make her appear dead for 42 hours. Romeo, believing Juliet to be dead, takes poison and dies in her tomb. When Juliet wakes up and finds Romeo dead, she also takes the same poison and dies. The tragic ending of the play reveals the devastating conseq

In [18]:
question = "How did Romeo meet Juliet?"
answer5 = ask_questions(rag_chain = rag_chain, question = question, chat_history_override = chat_history)


Question: How did Romeo meet Juliet?

Answer:  Romeo and Juliet met at a ball hosted by Capulet, where they fell in love at first sight. However, they were from feuding families, Montagues and Capulets, and their relationship was forbidden. Later, they met secretly with the help of Friar Lawrence, who married them in secret. After their marriage, they continued to meet secretly, and Friar Lawrence gave Juliet a potion to make it appear as if she was dead for 42 hours, so they could be apart without arousing suspicion. Romeo was to come to take her from the tomb when the potion's effect wore off, but he was unaware that Friar John, who was supposed to deliver the message, had been delayed. Tragically, both Romeo and Paris, who was betrothed to Juliet, ended up at the tomb on the same night, and a series of misunderstandings led to their deaths.



In [19]:
quiz = "How did Romeo meet his fate? go on to tell me how Juliet met her too."

answer6 = ask_questions(rag_chain = rag_chain, question = quiz, chat_history_override = chat_history)

Question: How did Romeo meet his fate? go on to tell me how Juliet met her too.

Answer:  Romeo met his fate by drinking poison, believing Juliet to be dead. Juliet met her fate by taking the same poison after finding Romeo dead in the tomb.



# Test for questions about the declaration of independence

In [20]:
question = "what grievances were made against the King of England?" 
answer7 = ask_questions(rag_chain = rag_chain, question = question)

Question: what grievances were made against the King of England?

Answer:  The grievances made against the King of England, as stated in the Declaration of Independence, include:

1. Refusing to give assent to laws that were necessary for the public good.
2. Forbidding governors to pass laws of immediate and pressing importance without suspending them until his assent was obtained, and then neglecting to attend to them.
3. Refusing to pass laws for the accommodation of large districts of people unless they would relinquish the right of representation in the legislature.
4. Calling legislative bodies together at places unusual, uncomfortable, and distant from the depository of their public records, for the sole purpose of fatiguing them into compliance with his measures.
5. Dissolving representative houses repeatedly for opposing his invasions on the rights of the people.
6. Refusing for a long time to cause others to be elected after dissolving the representative houses, leaving the le

In [21]:
question = "How did the declaration change the status of the American colonies?"

answer8 = ask_questions(rag_chain = rag_chain, question = question, chat_history_override = chat_history)


Question: How did the declaration change the status of the American colonies?

Answer:  The Declaration of Independence declared the American colonies to be free and independent states, absolving them from all allegiance to the British Crown and dissolving all political connection between them and Great Britain. This meant that the colonies were no longer under British rule and were now independent nations.



In [22]:
question = "What enlightenment ideas are embedded in the declaration of independence?"

answer9 = ask_questions(rag_chain = rag_chain, question = question, chat_history_override = chat_history)


Question: What enlightenment ideas are embedded in the declaration of independence?

Answer:  The Declaration of Independence, written by Thomas Jefferson, is a foundational document of the United States that reflects Enlightenment ideas. Some of the Enlightenment ideas found in the Declaration include:

1. Natural Rights: The Declaration asserts that all men are created equal and are endowed by their Creator with certain unalienable Rights, such as Life, Liberty, and the pursuit of Happiness. This idea is rooted in the Enlightenment belief in the inherent rights of individuals.

2. Consent of the Governed: The Declaration states that governments are instituted among men to secure these rights, deriving their just powers from the consent of the governed. This idea reflects the Enlightenment belief in the social contract, where the people have the right to choose their government and can alter or abolish it if it becomes destructive of their rights.

3. Reason and Rationality: The Decla

In [23]:
question = "What events led to the drafting of the declaration of independence in 1776?"

answer10 = ask_questions(rag_chain = rag_chain, question = question, chat_history_override = chat_history)

Question: What events led to the drafting of the declaration of independence in 1776?

Answer:  The events that culminated in the drafting of the Declaration of Independence in 1776 were a long series of abuses and usurpations by the King of Great Britain, which were aimed at establishing an absolute tyranny over the American colonies. These abuses included refusing to pass necessary laws, obstructing the administration of justice, raising taxes without consent, quartering troops among the colonists, and more. The colonists believed that these actions violated their rights to life, liberty, and the pursuit of happiness, and so they declared their independence from Great Britain.

